# Actividad 3: Entrenamiento de un modelo de Machine Learning (ML)

El proceso de entrenamiento de un modelo de ML consiste en proporcionar datos de entrenamiento, de los cuales aprende un algoritmo de ML (es decir, el algoritmo de aprendizaje). El término modelo de ML se refiere al artefacto de modelo que se crea en el proceso de entrenamiento. Los datos de entrenamiento se obtienen a través del proceso de ETL generado, el cual, debe contener la variable objetivo o tartet y cada de una de las variables que ayudan al modelo a encontrar la respuesta/salida correcta. Una vez los datos son obtenidos, inicial el proceso de entrenamiento. En un proceso de entrenamiento podemos encontrar diferentes pasos que varian dependiendo de la complejidad del problema, que son: preprocesamiento de datos, definir arquitectura del modelo, entrenar y evaluar el desempeño.

En la presente actividad entrenaremos un modelo de ML para identificar si un tipo de cáncer de mama puede ser **Maligno** o **Benigno**, la cual, vendría a ser nuestra variable objetivo o target. En el dataset, haremos uso de las siguientes características.

* radius: media de las distancias del centro a los puntos del perímetro
* texture: desviación estándar de los valores de la escala de grises
* perimeter
* area
* smoothness: variación local en longitudes de radio
* compactness:(perimeter^2 / area - 1.0)
* concavity: gravedad de las porciones cóncavas del contorno
* concave points: número de porciones cóncavas del contorno
* symmetry
* fractal dimension: (aproximación a la línea de costa" - 1)



# Imports 

Librerias que se necesitan para el desarrollo de la actividad

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Configurando pandas para visualización de 100 columnas

In [ ]:
pd.options.display.max_columns = 100

# Lectura del dataset

In [ ]:
path = 'https://raw.githubusercontent.com/eyberthrojas/Actividad_3_ML/main/is_cancer.csv'
data = pd.read_csv(path, on_bad_lines='skip')

In [ ]:
# Dimensión del dataset
data.shape

(569, 33)

In [ ]:
data.to_csv('is_cancer.csv', index=False)

In [ ]:
# Primeros 5 registros del dataset
data.head(5)

In [ ]:
# Inspeccionando la variable objetivo
data.diagnosis.value_counts()

In [ ]:
# Validando si hay presentes valores NULLs en el dataset
data.isnull().sum()

_**Generalmente, los IDs que identifican registros en un dataset no aportan información significativa al entrenamiento de un modelo de ML. Al igual que los valores NULL**_

In [ ]:
data.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

# Entrenamiento del modelo

_**Como se mencionó al inicio del notebook. Para esta actividad, solo utilizaremos algunas features del dataset completo**_

In [ ]:
features_finales = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", 
    "fractal_dimension_mean", "diagnosis"
]

In [ ]:
# Seleccionamos solo las features de interés del dataset completo
dataset_final = data[features_finales].copy(deep=True)

In [ ]:
# Ver tipos de datos de nuestro conjunto de datos
dataset_final.dtypes

Se puede observar que todas las features son de tipo númerico. En un proceso de entrenamiento de un modelo de ML nos podemos encontrar con otros tipos de datos, como los categóricos. Para efectos de creación del proceso de entrenamiento que nos ofrece Sklearn, crearemos una variable categórica. **Nota:** la menera de crear la variable categórica se realiza de aleatoria, sin embargo, en ciencia de datos existe un proceso llamado **Feature Engineering**, en donde se detallan otra maneras de realizrlo

In [ ]:
# Variable categórica creada a partir de la feature fractal_dimension_mean. 
dataset_final["calc_fractal_dimension_mean"] = np.where(
    1, dataset_final["fractal_dimension_mean"] < 0.055, 
    0
)

In [ ]:
# Eliminamos la variable númerica que usamos para crear la variable categórica
dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)

## Pipeline de Sklearn para el entrenamiento de un modelo de ML

[Sklearn](https://scikit-learn.org/stable/) ofrece una serie de pasos en los cuales podemos definir la arquitectura de nuestro modelo. En estos pasos, podemos definir el preprocesamiento de nuestras variables númericas y categóricas, realizar un feature unión y especificar el algoritmo de ML a usar


In [ ]:
# Clase que realiza el fit y transform a una variable
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

### Definicíon de pipeline variables númericas

In [ ]:
# Features numéricas
numerical = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean"
]

In [ ]:
# ColumnSelector: selecciona las columnas númericas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con la mediana de todos los valores de la feature
num_pipe = Pipeline([
    ('selector', ColumnSelector(numerical)),
    ('imputer', SimpleImputer(strategy='median'))
])

# Definicíon de pipeline variables categóricas

In [ ]:
# Features categóricas
categorical = ["calc_fractal_dimension_mean"]

In [ ]:
# ColumnSelector: selecciona las columnas categóricas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con un valor fijo
# OneHotEncoder: codificación de variables categóricas
cat_pipe = Pipeline([
    ('selector', ColumnSelector(categorical)),
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [ ]:
# Feature union de todas las features categóricas y numéricas
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

In [ ]:
# Pipeline final del proceso de entrenamiento
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svm.SVC())
])

In [ ]:
# Se elimina la variable target de los datos que contienen las features 
X = dataset_final.drop(['diagnosis'], axis = 1)

In [ ]:
# Se genera en una variable separada el target del problema, en este caso "Diagnosis"
y = dataset_final['diagnosis']

In [ ]:
# De todo el conjunto de datos se toman crean datos de train y test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
print("Size of training set:", X_train.shape)
print("Size of test set:", X_test.shape)

In [ ]:
# Entrenamiento
model = pipe.fit(X_train,y_train)

In [ ]:
# Predicción sobre los datos de test
y_pred = model.predict(X_test)

# Validación performance del modelo

Evaluamos el desempeño del modelo haciendo uso de la librería [Classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)  que otorga Sklearn. En esta tabla podemos encontrar diferentes métricas, en donde cada una se utiliza dependiendo del problema a resolver y como afecta una mala predicción por parte del modelo. Para esta actividad, vamos a usar el **Accuracy**, la cual, lo que nos muestra es el porcentaje de casos que el modelo ha acertado, en esta actividad, obtenemos un accuracy del **0.88**

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

# Ajuste de Hiperparámetros del modelo

Los hiperparámetros se definen como los parámetros definidos explícitamente por el usuario para controlar el proceso de aprendizaje. Cada algortimo de aprendizaje automático contiene hiperparámetros específicos que se seleccionan y optimizan durante el proceso de entrenamiento con el objetivo de mejorar el desempeño del modelo.

Para la actividad, utilizaremos ***GridSearchCV***, el cual, es una librería se [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) que realiza una búsqueda exhaustiva sobre valores de parámetros específicos para un estimador, en este caso, para el [Support Vector Machine (SVM)](https://scikit-learn.org/stable/modules/svm.html)


In [ ]:
# Definimos un rango de búsqueda de mejores hiperparámetros
parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

In [ ]:
# Búsqueda de mejores hiperaparámetros. Esta busqueda se realiza utilizando los datos de entrenamiento
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train,y_train)

In [ ]:
# Mejores hiperparametros encontrados por GridSearch
clf.best_params_

In [ ]:
# Generar el pipeline con los mejores hiperparametros
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', svm.SVC(C=10, kernel='linear'))
])

In [ ]:
# Entrenar nuevamente
model = pipe.fit(X_train, y_train)

In [ ]:
# Predecir nuevamente sobre los datos de test
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

Podemos observar que el accuracy aumenta del 0.88 al 0.91. Con esto, podemos notar que la optimización de los hiperparámetros del modelo es un paso que ayuda al modelo a tener un mejor desempeño en diferentes tipos de tareas, para esta actividad, una tarea centrada en la clasificación de dos tipos de cancer de mama